In [ ]:
# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Project imports
import twinlab as tl

In [ ]:
# Parameters
dataset_id = "data-error"
dataset_std_id = "data-error-std"
campaign_id = dataset_id
err_sig = 0.05
n = 20
random_seed = 42

In [ ]:
# Seed the random-number generator
np.random.seed(random_seed)

# Training Data
X = np.random.rand(n)
y = np.sin(X*2.*np.pi)+np.random.normal(0., err_sig, n)
df_train = pd.DataFrame({'X': X, 'y': y})
display(df_train)
tl.upload_dataset(df_train, dataset_id, verbose=True)

In [ ]:
# Evaluation data
eval = {"X": np.linspace(0, 1, 101)}
df_test = pd.DataFrame(eval)
display(df_test)

In [ ]:
# Training parameters
params = {
    "dataset_id": dataset_id,
    "inputs" : ["X"],
    "outputs": ["y"],
    "test_train_ratio": 1.,
}

# Plot parameters
alpha_fill = 0.5
nrow, ncol = 1, 3
figx, figy = 4, 3
grid = df_test["X"].values

# Loop over different error in data and plot
plt.subplots(nrow, ncol, figsize=(ncol*figx, nrow*figy))
for iplot in range(3):

    # (Heteroscedastic) error in data 'X' and upload
    if iplot == 0:
        df_std = None
        label = "Learned noise"
    elif iplot == 1:
        df_std = pd.DataFrame({'y': np.zeros(n)})#, 'y': np.zeros(n)})
        tl.upload_dataset(df_std, dataset_std_id, verbose=True)
        label = "Noise fixed to zero"
    elif iplot == 2:
        df_std = pd.DataFrame({'y': 0.05*np.ones(n)})#, 'y': np.zeros(n)})
        label = "Noise fixed to true value"
    else:
        raise ValueError("Invalid iplot")
    print(type(df_std))
    display(df_std)
    if iplot in [1, 2]:
        tl.upload_dataset(df_std, dataset_std_id, verbose=True)
        params["dataset_std_id"] = dataset_id

    # Train model
    tl.train_campaign(params, campaign_id, verbose=True)
    
    # Predict from model
    df_mean, df_std = tl.predict_campaign(df_test, campaign_id)
    mean, err = df_mean["y"].values, df_std["y"].values
    color = f"C{iplot}"
    
    # Plot
    plt.subplot(nrow, ncol, iplot+1)
    plt.plot(df_train["X"][:n], df_train["y"][:n], ".", color="black")
    plt.plot(grid, mean, "-", color=color, label=label)
    for nsig in [1, 2]:
        plt.fill_between(grid, mean-nsig*err, mean+nsig*err, lw=0, color=color, alpha=alpha_fill)
    # plt.xlabel("X"); plt.ylabel("y")
    plt.xticks([]); plt.yticks([])
    plt.ylim((-1.5, 1.5))
    plt.legend()

# Finalize plot
plt.tight_layout()
plt.show()

In [ ]:
# Delete campaign and dataset if necessary
# tl.delete_campaign(campaign_id, verbose=True)
# tl.delete_dataset(dataset_id, verbose=True)